In [ ]:
import cv2
import numpy as np
import pyautogui
from tensorflow.keras.models import load_model

# global variables
bg = None

def run_avg(image, accumWeight):
    global bg
    if bg is None:
        bg = image.copy().astype("float")
        return

    cv2.accumulateWeighted(image, bg, accumWeight)

def segment(image, threshold=25):
    global bg
    diff = cv2.absdiff(bg.astype("uint8"), image)
    thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)[1]
    (cnts, _) = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(cnts) == 0:
        return
    else:
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)

def _load_weights():
    try:
        model = load_model("model_cnn.h5")
        print(model.summary())
        return model
    except Exception as e:
        return None

def getPredictedClass(model):
    if model is None:
        print("Model is not loaded successfully.")
        return None

    image = cv2.imread('Temp.png')
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_image = cv2.resize(gray_image, (100, 100))
    gray_image = np.expand_dims(gray_image, axis = 0)
    gray_image = gray_image.astype(np.float32)
    gray_image = gray_image / 255.0

    prediction = model.predict_on_batch(gray_image)

    predicted_class = np.argmax(prediction)
    classes = ["Blank", "OK", "Thumbs Up", "Thumbs Down", "Punch", "High Five"]
    return classes[predicted_class]

gesture_to_keys = {"Blank":"No Key","OK":"A", "Thumbs Up":"S", "Thumbs Down":"W", "Punch":"D", "High Five":"Space"}
if __name__ == "__main__":
    accumWeight = 0.5
    camera = cv2.VideoCapture(0)

    fps = int(camera.get(cv2.CAP_PROP_FPS))
    top, right, bottom, left = 10, 350, 225, 590
    num_frames = 0
    calibrated = False
    model = _load_weights()
    k = 0

    while True:
        (grabbed, frame) = camera.read()
        frame = cv2.resize(frame, (700, 700))
        frame = cv2.flip(frame, 1)
        clone = frame.copy()
        (height, width) = frame.shape[:2]
        roi = frame[top:bottom, right:left]
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        if num_frames < 30:
            run_avg(gray, accumWeight)
            if num_frames == 1:
                print("[STATUS] please wait! calibrating...")
            elif num_frames == 29:
                print("[STATUS] calibration successful...")
        else:
            hand = segment(gray)
            if hand is not None:
                (thresholded, segmented) = hand
                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))

                if k % (fps / 6) == 0:
                    cv2.imwrite('Temp.png', thresholded)
                    predictedClass = getPredictedClass(model)
                    cv2.putText(clone, str(predictedClass), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    cv2.putText(clone, gesture_to_keys[str(predictedClass)], (70, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                    if predictedClass == "OK":
                        pyautogui.press('A')
                        print("A")
                    elif predictedClass == "Thumbs Up":
                        pyautogui.press('S')
                        print("S")
                    elif predictedClass == "Thumbs Down":
                        pyautogui.press('W')
                        print("W")
                    elif predictedClass == "Punch":
                        pyautogui.press('D')
                        print("D")
                    elif predictedClass == "High Five":
                        pyautogui.press('space')
                        print("Space")

                thresholded = cv2.cvtColor(thresholded, cv2.COLOR_GRAY2BGR)
                thresholded = cv2.resize(thresholded, (clone.shape[1], clone.shape[0]))  # Resize to match clone
                combined = np.hstack((clone, thresholded))

                # Draw the rectangle around the hand region
                cv2.rectangle(combined, (left, top), (right, bottom), (0, 255, 0), 2)

                cv2.imshow("Video Feed", combined)

        k += 1

        num_frames += 1

        keypress = cv2.waitKey(1) & 0xFF
        if keypress == ord("q"):
            break

    camera.release()
    cv2.destroyAllWindows()
